In [1]:
import os

In [2]:
os.chdir("C:/Users/athar/Projects/Deep Learning/Chicken-Disease-Classification")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) 
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    #Constructor with two optional arguments:
    #config_filepath: Path to your config.yaml.
    #params_filepath: Path to your params.yaml.
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):
        #Reads the config.yaml file and stores it in self.config as a Python dictionary
        self.config= read_yaml(config_filepath)
        #Reads the params.yaml file and stores it in self.params as a dictionary.
        self.params= read_yaml(params_filepath)
        #Creates the base directory (e.g., artifacts/) specified in config.yaml under the key artifacts_root.
        create_directories([self.config.artifacts_root], verbose=True)
        
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        
        create_directories([config.root_dir], verbose=True)
        
        data_ingestion_config= DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from CNNClassifier import logger
#from CNNClassifier.utils.common import get_size

In [9]:
class dataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"file already exsits on path: {(self.config.local_data_file)}")
            
            
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    di_config= ConfigurationManager() # create object of class
    data_ingestion_config= di_config.get_data_ingestion_config() #call function and store in other object
    data_ingestion=dataIngestion(config=data_ingestion_config) #create object for class
    data_ingestion.download_file() # call function
    data_ingestion.extract_zip_file() #call function
except Exception as e:
    raise e

[2025-05-12 20:33:22,182: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-12 20:33:22,183: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-12 20:33:22,185: INFO: common: Created directory at: artifacts]
[2025-05-12 20:33:22,187: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-05-12 20:33:24,534: INFO: 1916319891: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9D27:1FBF5D:1F1F0C:257F89:68229355
Accept-Ranges: bytes
Date: Tue, 13 May 2025 00:33:26 GMT
Via: 1.1 varnish
X-Served-By: cache-mia-kmia1760076-MIA
X